# PHMSA Data Analysis

## Safety Moment
https://www.phmsa.dot.gov/news/phmsa-recognizes-april-national-safe-digging-month

## 0 - Imports and Notebook Setup

In [ ]:
# import numpy and pandas
import numpy as np
import pandas as pd

# import datetime packages
import datetime
from datetime import datetime, date

# imports for downloading zip files
from zipfile import ZipFile
from urllib.request import urlopen
import os

# import html utilities
from IPython.display import IFrame
from IPython.display import FileLink

In [ ]:
# import matplotlib, seaborn, and plotnine
import matplotlib.pyplot as plt
import seaborn as sns
import plotnine #a ggplot-like package for python
%matplotlib inline

In [ ]:
# Customizations
sns.set() # set seaborn defaults defaults

plt.rcParams['figure.figsize'] = (12, 8)
plt.style.use('ggplot')

In [ ]:
# set prefix and directory for figures
fig_date = str(datetime.now()).split()[0]
fig_prefix = f"figures/{fig_date}-"
if not os.path.exists('figures/'):
    os.makedirs('figures/')

## 1 - Obtain Data
### Download data and read into pandas dataframe and export to csv

#### Data Source Website

https://www.phmsa.dot.gov/data-and-statistics/pipeline/source-data

https://www.phmsa.dot.gov/data-and-statistics/pipeline/distribution-transmission-gathering-lng-and-liquid-accident-and-incident-data

In [ ]:
url_2010_present = 'https://www.phmsa.dot.gov/sites/phmsa.dot.gov/files/data_statistics/pipeline/accident_hazardous_liquid_jan2010_present.zip'

In [ ]:
url = urlopen(url_2010_present)

In [ ]:
if not os.path.exists('../temp_files/'):
    os.makedirs('../temp_files/')

In [ ]:
output = open('../temp_files/zipFile.zip','wb')
output.write(url.read())
output.close()

In [ ]:
zip_obj = ZipFile('../temp_files/zipFile.zip', 'r')

# https://docs.python.org/3/library/zipfile.html#zipfile.ZipFile.extract

In [ ]:
data_file = zip_obj.extract(zip_obj.filelist[0],path="../temp_files")

In [ ]:
incid = pd.read_table(data_file, encoding='windows-1252', parse_dates=['LOCAL_DATETIME'])

In [ ]:
if not os.path.exists('proc_data/'):
    os.makedirs('proc_data/')

In [ ]:
incid.to_csv('proc_data/zip_to_csv_incidents_2010_present.csv', index=False)

### Get key to data

In [ ]:
data_key = zip_obj.extract(zip_obj.filelist[1])

In [ ]:
data_key

## 2 - Wrangle Data

In [ ]:
incid.shape

In [ ]:
incid.columns

### Select columns for exploration
Based on a review of the data_key, the following columns are relavent to the cause, location, and volume of releases:
                    `['LOCAL_DATETIME','NAME','ONSHORE_CITY_NAME','ONSHORE_STATE_ABBREVIATION',
                      'REPORT_TYPE','ON_OFF_SHORE','CAUSE','CAUSE_DETAILS','EX_PARTY_TYPE',
                      STRESS_SUBTYPE',
                      'STRESS_DETAILS','SYSTEM_PART_INVOLVED','ITEM_INVOLVED',
                      'LOCATION_LATITUDE', 
                      'LOCATION_LONGITUDE', 'COMMODITY_RELEASED_TYPE', 'UNINTENTIONAL_RELEASE_BBLS', 
                      'RECOVERED_BBLS','NARRATIVE']`

In [ ]:
incident_vol = incid[['LOCAL_DATETIME','NAME','ONSHORE_CITY_NAME','ONSHORE_STATE_ABBREVIATION',
                      'REPORT_TYPE','ON_OFF_SHORE','CAUSE','CAUSE_DETAILS','EX_PARTY_TYPE','STRESS_SUBTYPE',
                      'STRESS_DETAILS','SYSTEM_PART_INVOLVED','ITEM_INVOLVED','LOCATION_LATITUDE', 
                      'LOCATION_LONGITUDE', 'COMMODITY_RELEASED_TYPE', 'UNINTENTIONAL_RELEASE_BBLS', 
                      'RECOVERED_BBLS','NARRATIVE']]
incident_vol.head(3)

In [ ]:
incident_vol['NAME'].nunique()

### Filter DataFrame

In [ ]:
# Create filter to remove releases associated with Highly Volatile Liquids (HVL's), so we can focus on liquid (e.g., oil) and CO2
mask_no_hvl = ~incident_vol['COMMODITY_RELEASED_TYPE'].str.contains('HVL ')

In [ ]:
incid_vol_liquid_and_co2 = incident_vol[mask_no_hvl]

In [ ]:
# Create filter to remove CO2, so we can focus on liquids
mask_no_co2 = ~incid_vol_liquid_and_co2['COMMODITY_RELEASED_TYPE'].str.contains('CO2')
incid_vol_liquid = incid_vol_liquid_and_co2[mask_no_co2]

### Check data types in differnt columns and missing data

In [ ]:
incid_vol_liquid.info()

## 3 - Explore Data

In [ ]:
# Number of operators with "liquid" incidents from January 2010 to Present
incid_vol_liquid['NAME'].nunique()

In [ ]:
# Top 10 operators with the most frequent incidents since January 2010
incid_vol_liquid['NAME'].value_counts()[:10]

### Aggregation of released versus recovered volumes of oil

In [ ]:
# Total volume of unintentional releases since January 2010
released_bbl = incid_vol_liquid['UNINTENTIONAL_RELEASE_BBLS'].sum()
print(f'Total of {released_bbl} bbl of oil released since January 2010\n')
print(f'This is {released_bbl * 42} gallons')

In [ ]:
# Total volume of the released oil recovered since January 2010
recovered_bbl = incid_vol_liquid['RECOVERED_BBLS'].sum()
print(f'Total of {recovered_bbl} bbl of the released oil IMMEDIATELY recovered\n')
print(f'This is {recovered_bbl * 42} gallons or {recovered_bbl / released_bbl * 100} percent')

In [ ]:
# Vizualize the released vs recovered data
plt.scatter(incid_vol_liquid['UNINTENTIONAL_RELEASE_BBLS'], incid_vol_liquid['RECOVERED_BBLS'])
plt.xlabel('volume released (bbls)')
plt.ylabel('volume recovered (bbls)')
plt.title('PHMSA Reported Pipeline Incidents 2010 - Present\n(Liquid Commodity)')
plt.savefig(fig_prefix + "liquid-released-v-recovered.png", dpi=350) 

In [ ]:
# look at outliers > 20000 bbl release
mask_gt20000 = incid_vol_liquid['UNINTENTIONAL_RELEASE_BBLS'] > 20000

In [ ]:
large_releases = incid_vol_liquid[mask_gt20000]

In [ ]:
# DELETE? large_releases.reset_index(inplace=True)

In [ ]:
large_releases

In [ ]:
large_releases.info()

In [ ]:
# get coordinates for large releases and save to csv.
large_release_coord = large_releases[['NAME', 'LOCAL_DATETIME', 
                                      'LOCATION_LATITUDE', 'LOCATION_LONGITUDE']]
large_release_coord.to_csv('proc_data/large_release_coord.csv')

### Import large release coordinates into google earth pro
Launch google earth

### More Visualizations

In [ ]:
# Look at distribution of percent recovery
incid_vol_liquid['PERCENT_RECOV'] = (incid_vol_liquid['RECOVERED_BBLS'] / incid_vol_liquid['UNINTENTIONAL_RELEASE_BBLS']) * 100
incid_vol_liquid['PERCENT_RECOV'].head()

In [ ]:
# Histogram of percent recovery
incid_vol_liquid['PERCENT_RECOV'].plot(kind = 'hist', bins = 10)
plt.xlabel('percent recovery')
plt.ylabel('number of incidents')
plt.title('PHMSA Reported Pipeline Incidents 2010 - Present\n(Liquid Commodity)')
plt.savefig(fig_prefix + "liquid-percent-recovery.png", dpi=350)

Questions to further explore for incidents with less than 10% recovery:
- what was the cause of the release?
- is low recovery related to certain operators?
- is low recovery related to location of release?
- ??

In [ ]:
# Scatter plot of percent recovery versus volume released
plt.scatter(incid_vol_liquid['UNINTENTIONAL_RELEASE_BBLS'], incid_vol_liquid['PERCENT_RECOV'])
plt.xlabel('volume released (bbls)')
plt.ylabel('percent recovered')
plt.title('PHMSA Reported Pipeline Incidents 2010 - Present\n(Liquid Commodity)')
plt.savefig(fig_prefix + "liquid-percent-recovered-by-vol-released.png", dpi=350)

#### Don't hate me because I use a pie-chart...

In [ ]:
incid_vol_liquid['CAUSE'].value_counts().plot(kind = 'pie', legend=False)
plt.savefig(fig_prefix + "major-incident-causes-pie.png", dpi=350)

In [ ]:
incid_vol_liquid.columns

In [ ]:
incid_vol_liquid[['CAUSE', 'CAUSE_DETAILS']].head(3)

In [ ]:
causes = incid_vol_liquid['CAUSE_DETAILS'].value_counts().sort_values(ascending=False)
causes.head()

In [ ]:
# sns.barplot(causes.values, causes.index)
causes.plot.barh()
plt.xlabel('number of incidents')
plt.title('Causes of PHMSA Reported Pipeline Incidents\n(2010 - Present for Liquid Commodity)')
plt.savefig(fig_prefix + "frequency-of-causes.png", dpi=350)

In [ ]:
# Explore causes from EXCAVATION DAMAGE BY THIRD PARTY.
mask_3rd_party = incid_vol_liquid['CAUSE_DETAILS'] == 'EXCAVATION DAMAGE BY THIRD PARTY'
incid_3rd_party_excav = incid_vol_liquid[mask_3rd_party]

In [ ]:
# Plot histogram of volume of release caused by 3rd party excavation.
incid_3rd_party_excav['UNINTENTIONAL_RELEASE_BBLS'].plot(kind = 'hist', bins = 10)
plt.xlabel('volume of release')
plt.ylabel('number of incidents')
plt.title('PHMSA Reported Pipeline Incidents 2010 - Present\n(Liquid Commodity)')
plt.savefig(fig_prefix + "release_3rd_party_excavation.png", dpi=350)

In [ ]:
# Explore incidents related from internal corrosion
mask_int_corr = incid_vol_liquid['CAUSE_DETAILS'] == 'INTERNAL CORROSION'

In [ ]:
incid_ic = incid_vol_liquid[mask_int_corr]

In [ ]:
incid_ic['UNINTENTIONAL_RELEASE_BBLS'].sum()

In [ ]:
# Create a dictionary of causes versus total volume of releases
causes_dict = {}
for item in causes.index:
    item_bbl_sum = incid_vol_liquid[incid_vol_liquid['CAUSE_DETAILS'] == item]['UNINTENTIONAL_RELEASE_BBLS'].sum()
    print('Cause {} resulted in total releases of {} bbls from 2010 to present'.format(item, item_bbl_sum))
    causes_dict[item] = item_bbl_sum

In [ ]:
causes_dict

In [ ]:
causes_series = pd.Series(causes_dict).sort_values(ascending=True)
causes_series.tail()

In [ ]:
# Plot total volume releassed by cause
causes_series.plot.barh()
plt.xlabel('total volume released (bbls)')
plt.title('Causes of PHMSA Reported Pipeline Incidents\n(2010 - Present for Liquid Commodity)')
plt.savefig(fig_prefix + "liquid-released-by-cause.png", dpi=350)

In [ ]:
# Explore average release volume related to each cause
causes_rate_dict = {}
for item in causes.index:
    item_bbl_sum = incid_vol_liquid[incid_vol_liquid['CAUSE_DETAILS'] == item]['UNINTENTIONAL_RELEASE_BBLS'].sum()
    item_count = causes[item]
    print('Cause {} resulted in averaage of {} bbls per release from 2010 to present'.format(item, item_bbl_sum))
    causes_rate_dict[item] = item_bbl_sum / item_count

In [ ]:
causes_rate_series = pd.Series(causes_rate_dict).sort_values(ascending=True)
causes_rate_series.tail()

In [ ]:
causes_rate_series.plot.barh()
plt.xlabel('total volume released PER incident (bbls)')
plt.title('Causes of PHMSA Reported Pipeline Incidents\n(2010 - Present for Liquid Commodity)')
plt.savefig(fig_prefix + "liquid-released-per-incident-by-cause.png", dpi=350)

In [ ]:
# Further explore causes related to environmetnal cracking-related
mask_env_crack = incid_vol_liquid['CAUSE_DETAILS'] == 'ENVIRONMENTAL CRACKING-RELATED'
env_cracking = incid_vol_liquid[mask_env_crack]

In [ ]:
env_cracking[['NAME', 'STRESS_SUBTYPE', 'STRESS_DETAILS', 'UNINTENTIONAL_RELEASE_BBLS']]

## 4 - Generate Word Clouds

TODO: Generate word clouds from notes columns

In [ ]:
narr_text = ''
for text in env_cracking['NARRATIVE']:
    narr_text = narr_text + ' ' + str(text)

In [ ]:
from subprocess import check_output
from wordcloud import WordCloud, STOPWORDS
import matplotlib as mpl

In [ ]:
mpl.rcParams['font.size']=12                
mpl.rcParams['savefig.dpi']=100             
mpl.rcParams['figure.subplot.bottom']=.1

In [ ]:
stopwords = set(STOPWORDS)

In [ ]:
wordcloud = WordCloud(
                          background_color='white',
                          stopwords=stopwords,
                          max_words=200,
                          max_font_size=40, 
                          random_state=42
                         ).generate(str(narr_text))

In [ ]:
print(wordcloud)
fig = plt.figure(1)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()
fig.savefig("word1.png", dpi=900)

### EXTRA - Export large releases to geojson file

In [ ]:
import json
import geojson

In [ ]:
##### SOMETHING NOT WORKING RIGHT

In [ ]:
def data2geojson(df):
    points = []
    df.apply(lambda X: points.append((X[["LOCATION_LONGITUDE"]],
                                     X[["LOCATION_LATITUDE"]]), axis=1))
    with open('map.geojson', 'w') as fp:
        geojson.dump(geojson.MultiPoint(points), fp, sort_keys=True)

In [ ]:
large_releases[['LOCATION_LONGITUDE', 'LOCATION_LATITUDE']]

In [ ]:
with open('map.geojson', 'w') as fp:
    geojson.dump(geojson.MultiPoint([(-102.856912,48.524251),(-84.972510,42.243290)]), fp, sort_keys=True)